In [ ]:
import model_utils as mu
mu.DEATH_RATE = 1.0
mu.ICU_RATE = 5.0
mu.HOSPITAL_RATE = 15.0
mu.SYMPTOM_RATE = 20.0
mu.INFECT_2_HOSPITAL_TIME = 12
mu.HOSPITAL_2_ICU_TIME = 2
mu.ICU_2_DEATH_TIME = 5
mu.ICU_2_RECOVER_TIME = 11
mu.NOT_ICU_DISCHARGE_TIME = 7

In [ ]:
log_pre, mb = mu.get_log_daily_predicted_death_by_country('Singapore', lockdown_date='20200415')

In [ ]:
daily_metrics, cum_metrics, model_beta = mu.get_metrics_by_country('US', lockdown_date='20200317')

In [ ]:
daily_metrics.plot()

In [ ]:
import plotly.offline as py_offline
import cufflinks as cf
cf.go_offline()
py_offline.__PLOTLY_OFFLINE_INITIALIZED = True

In [ ]:
daily_metrics.iplot()